In [1]:
import pandas as pd
import numpy as np
import re
import os
import Levenshtein as lev #might not need this
from EcoNameTranslator import to_scientific
from EcoNameTranslator import to_common #might not need this
import requests
from bs4 import BeautifulSoup as bs

In [2]:
fish_acnh = pd.read_csv("fish.csv")

In [4]:
fish_acnh.head()

,#,Name,Sell,Where/How,Shadow,Total Catches to Unlock,Spawn Rates,Rain/Snow Catch Up,NH Jan,NH Feb,...,SH Dec,Color 1,Color 2,Size,Lighting Type,Icon Filename,Critterpedia Filename,Furniture Filename,Internal ID,Unique Entry ID
0,56,anchovy,200,Sea,Small,0,2–5,No,4 AM – 9 PM,4 AM – 9 PM,...,4 AM – 9 PM,Blue,Red,1x1,No lighting,Fish81,FishAntyobi,FtrFishAntyobi,4201,LzuWkSQP55uEpRCP5
1,36,angelfish,3000,River,Small,20,2–5,No,NaN,NaN,...,4 PM – 9 AM,Yellow,Black,1x1,Fluorescent,Fish30,FishAngelfish,FtrFishAngelfish,2247,XTCFCk2SiuY5YXLZ7
2,44,arapaima,10000,River,XX-Large,50,1,Yes,NaN,NaN,...,4 PM – 9 AM,Black,Blue,3x2,No lighting,Fish36,FishPiraruku,FtrFishPiraruku,2253,mZy4BES54bqwi97br
3,41,arowana,10000,River,Large,50,1–2,No,NaN,NaN,...,4 PM – 9 AM,Yellow,Black,2x1,Fluorescent,Fish33,FishArowana,FtrFishArowana,2250,F68AvCaqddBJL7ZSN
4,58,barred knifejaw,5000,Sea,Medium,20,3–5,No,NaN,NaN,...,All day,White,Black,1x1,Fluorescent,Fish47,FishIshidai,FtrFishIshidai,2265,X3R9SFSAaDzBF4fE3


In [5]:
insects_acnh = pd.read_csv('insects.csv')

Need to remove these rows :wikipedia:Japanese language,wikipedia:Hepburn romanization,de:nachbarn/Armin

In [6]:
import re

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [8]:
critters_acnh = pd.concat([insects_acnh,fish_acnh], ignore_index=True)

In [9]:
critters_acnh.head()

,#,Name,Sell,Where/How,Weather,Total Catches to Unlock,Spawn Rates,NH Jan,NH Feb,NH Mar,NH Apr,NH May,NH Jun,NH Jul,NH Aug,NH Sep,NH Oct,NH Nov,NH Dec,SH Jan,SH Feb,SH Mar,SH Apr,SH May,SH Jun,SH Jul,SH Aug,SH Sep,SH Oct,SH Nov,SH Dec,Color 1,Color 2,Icon Filename,Critterpedia Filename,Furniture Filename,Internal ID,Unique Entry ID,Shadow,Rain/Snow Catch Up,Size,Lighting Type
0,10,agrias butterfly,3000,Flying near flowers,Any except rain,20,5,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,Pink,Green,Ins6,InsectMiirotateha,FtrInsectMiirotateha,620,aj95rMzdbSbvZy9A2,NaN,NaN,NaN,NaN
1,69,ant,80,On rotten turnips,Any weather,0,0,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,Black,White,Ins26,InsectAri,FtrInsectAri,588,QZpmczZu4hW2a4Rpv,NaN,NaN,NaN,NaN
2,14,Atlas moth,3000,On trees (any kind),Any weather,20,5,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,NaN,NaN,NaN,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,Orange,Yellow,Ins10,InsectYonagunisan,FtrInsectYonagunisan,652,u2GhYQJXDCQKp7AQ8,NaN,NaN,NaN,NaN
3,68,bagworm,600,Shaking trees (hardwood or cedar only),Any weather,0,50,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,Brown,Blue,Ins36,InsectMinomushi,FtrInsectMinomushi,622,QvxgCm82JqHsDknY4,NaN,NaN,NaN,NaN
4,34,banded dragonfly,4500,Flying near water,Any except rain,50,7,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,Black,Yellow,Ins24,InsectOniyanma,FtrInsectOniyanma,635,pCFep58D6QusMSvR7,NaN,NaN,NaN,NaN


In [10]:
critters_acnh = pd.concat([insects_acnh,fish_acnh], ignore_index=True)

In [11]:
critters_acnh.head()

,#,Name,Sell,Where/How,Weather,Total Catches to Unlock,Spawn Rates,NH Jan,NH Feb,NH Mar,NH Apr,NH May,NH Jun,NH Jul,NH Aug,NH Sep,NH Oct,NH Nov,NH Dec,SH Jan,SH Feb,SH Mar,SH Apr,SH May,SH Jun,SH Jul,SH Aug,SH Sep,SH Oct,SH Nov,SH Dec,Color 1,Color 2,Icon Filename,Critterpedia Filename,Furniture Filename,Internal ID,Unique Entry ID,Shadow,Rain/Snow Catch Up,Size,Lighting Type
0,10,agrias butterfly,3000,Flying near flowers,Any except rain,20,5,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,Pink,Green,Ins6,InsectMiirotateha,FtrInsectMiirotateha,620,aj95rMzdbSbvZy9A2,NaN,NaN,NaN,NaN
1,69,ant,80,On rotten turnips,Any weather,0,0,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,Black,White,Ins26,InsectAri,FtrInsectAri,588,QZpmczZu4hW2a4Rpv,NaN,NaN,NaN,NaN
2,14,Atlas moth,3000,On trees (any kind),Any weather,20,5,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,NaN,NaN,NaN,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,Orange,Yellow,Ins10,InsectYonagunisan,FtrInsectYonagunisan,652,u2GhYQJXDCQKp7AQ8,NaN,NaN,NaN,NaN
3,68,bagworm,600,Shaking trees (hardwood or cedar only),Any weather,0,50,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,Brown,Blue,Ins36,InsectMinomushi,FtrInsectMinomushi,622,QvxgCm82JqHsDknY4,NaN,NaN,NaN,NaN
4,34,banded dragonfly,4500,Flying near water,Any except rain,50,7,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,Black,Yellow,Ins24,InsectOniyanma,FtrInsectOniyanma,635,pCFep58D6QusMSvR7,NaN,NaN,NaN,NaN


In [12]:
critters_acnh["Name2"] = critters_acnh["Name"].str.title()
critters_acnh["Name2"] = critters_acnh["Name2"].str.replace(" ","_")
critters_acnh["Name2"] = critters_acnh["Name2"].str.replace("'S","%27s")
critters_acnh["Name2"] = critters_acnh["Name2"].str.replace("Anchovy", "Anchovy_(fish)")
critters_acnh["Name2"] = critters_acnh["Name2"].str.replace("Frog", "Frog_(fish)")

In [13]:
critters_acnh.head()

,#,Name,Sell,Where/How,Weather,Total Catches to Unlock,Spawn Rates,NH Jan,NH Feb,NH Mar,NH Apr,NH May,NH Jun,NH Jul,NH Aug,NH Sep,NH Oct,NH Nov,NH Dec,SH Jan,SH Feb,SH Mar,SH Apr,SH May,SH Jun,SH Jul,SH Aug,SH Sep,SH Oct,SH Nov,SH Dec,Color 1,Color 2,Icon Filename,Critterpedia Filename,Furniture Filename,Internal ID,Unique Entry ID,Shadow,Rain/Snow Catch Up,Size,Lighting Type,Name2
0,10,agrias butterfly,3000,Flying near flowers,Any except rain,20,5,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,Pink,Green,Ins6,InsectMiirotateha,FtrInsectMiirotateha,620,aj95rMzdbSbvZy9A2,NaN,NaN,NaN,NaN,Agrias_Butterfly
1,69,ant,80,On rotten turnips,Any weather,0,0,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,Black,White,Ins26,InsectAri,FtrInsectAri,588,QZpmczZu4hW2a4Rpv,NaN,NaN,NaN,NaN,Ant
2,14,Atlas moth,3000,On trees (any kind),Any weather,20,5,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,NaN,NaN,NaN,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,Orange,Yellow,Ins10,InsectYonagunisan,FtrInsectYonagunisan,652,u2GhYQJXDCQKp7AQ8,NaN,NaN,NaN,NaN,Atlas_Moth
3,68,bagworm,600,Shaking trees (hardwood or cedar only),Any weather,0,50,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,Brown,Blue,Ins36,InsectMinomushi,FtrInsectMinomushi,622,QvxgCm82JqHsDknY4,NaN,NaN,NaN,NaN,Bagworm
4,34,banded dragonfly,4500,Flying near water,Any except rain,50,7,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,Black,Yellow,Ins24,InsectOniyanma,FtrInsectOniyanma,635,pCFep58D6QusMSvR7,NaN,NaN,NaN,NaN,Banded_Dragonfly


In [14]:
critlist6 = critters_acnh['Name2'].tolist()

In [15]:
crittersname6=[]
critterslink6=[]
critterstitle6=[]


for k in critlist6:
    base_urlj = "https://nookipedia.com/wiki/"
    urlj = base_urlj+k
    critterslink6.append(urlj)
    response = requests.get(urlj)
    soup3 = bs(response.content, 'lxml')
    j = (soup3.find(text="Name: ").findNext('i').contents)
    crittersname6.append(k)
    critterstitle6.append(j)
    print(j)

[<a class="extiw" href="https://en.wikipedia.org/wiki/Agrias_claudina" title="wikipedia:Agrias claudina">Agrias claudina sardanapalus</a>]
['Unknown']
['Attacus atlas']
['Metura elongatus']
[<a class="extiw" href="https://en.wikipedia.org/wiki/Anotogaster_sieboldii" title="wikipedia:Anotogaster sieboldii">Anotogaster sieboldii</a>]
['Homoeogryllus japonica']
['Eupholus magnificus']
['Graptopsaltria nigrofuscata']
['Lithobius forficatus']
['Unknown']
['Anoplophora chinensis']
[<a class="extiw" href="https://en.wikipedia.org/wiki/Graphium_sarpedon" title="wikipedia:Graphium sarpedon">Graphium sarpedon</a>]
[<a class="extiw" href="https://en.wikipedia.org/wiki/Pieris_rapae" title="wikipedia:Pieris rapae">Pieris rapae</a>]
['Acheta domesticus']
['Cyclommatus elaphus']
['Ischnura heterosticta']
[<a class="extiw" href="https://en.wikipedia.org/wiki/Anax_parthenope" title="wikipedia:Anax parthenope">Anax parthenope</a>]
['Dytiscus marginalis']
['Pseudotorynorrhina japonica']
['Copris pecuariu

In [16]:
len(critterslink6)

160

In [17]:
critters = pd.DataFrame(list(zip(crittersname6, critterslink6, critterstitle6)), 
                                   columns = ["Name", "Link",'title'])

In [18]:
critters.head()

,Name,Link,title
0,Agrias_Butterfly,https://nookipedia.com/wiki/Agrias_Butterfly,[[Agrias claudina sardanapalus]]
1,Ant,https://nookipedia.com/wiki/Ant,[Unknown]
2,Atlas_Moth,https://nookipedia.com/wiki/Atlas_Moth,[Attacus atlas]
3,Bagworm,https://nookipedia.com/wiki/Bagworm,[Metura elongatus]
4,Banded_Dragonfly,https://nookipedia.com/wiki/Banded_Dragonfly,[[Anotogaster sieboldii]]


In [19]:
critters['sci_name'] = critters["title"].map(lambda x: 0 if len(x) ==0 else x[0])

In [20]:
critters.head()

,Name,Link,title,sci_name
0,Agrias_Butterfly,https://nookipedia.com/wiki/Agrias_Butterfly,[[Agrias claudina sardanapalus]],[Agrias claudina sardanapalus]
1,Ant,https://nookipedia.com/wiki/Ant,[Unknown],Unknown
2,Atlas_Moth,https://nookipedia.com/wiki/Atlas_Moth,[Attacus atlas],Attacus atlas
3,Bagworm,https://nookipedia.com/wiki/Bagworm,[Metura elongatus],Metura elongatus
4,Banded_Dragonfly,https://nookipedia.com/wiki/Banded_Dragonfly,[[Anotogaster sieboldii]],[Anotogaster sieboldii]


In [21]:
critters_1= critters[['Name','title','sci_name']].copy()
critters_1.to_csv('critters_chk.csv',index=False)

In [22]:
critters_1.head()

,Name,title,sci_name
0,Agrias_Butterfly,[[Agrias claudina sardanapalus]],[Agrias claudina sardanapalus]
1,Ant,[Unknown],Unknown
2,Atlas_Moth,[Attacus atlas],Attacus atlas
3,Bagworm,[Metura elongatus],Metura elongatus
4,Banded_Dragonfly,[[Anotogaster sieboldii]],[Anotogaster sieboldii]


In [23]:
critters_acnh_sci = critters_acnh.merge(critters_1, left_on = "Name2", right_on="Name", how = "left")

In [24]:
critters_acnh_sci.head()

,#,Name_x,Sell,Where/How,Weather,Total Catches to Unlock,Spawn Rates,NH Jan,NH Feb,NH Mar,NH Apr,NH May,NH Jun,NH Jul,NH Aug,NH Sep,NH Oct,NH Nov,NH Dec,SH Jan,SH Feb,SH Mar,SH Apr,SH May,SH Jun,SH Jul,SH Aug,SH Sep,SH Oct,SH Nov,SH Dec,Color 1,Color 2,Icon Filename,Critterpedia Filename,Furniture Filename,Internal ID,Unique Entry ID,Shadow,Rain/Snow Catch Up,Size,Lighting Type,Name2,Name_y,title,sci_name
0,10,agrias butterfly,3000,Flying near flowers,Any except rain,20,5,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,Pink,Green,Ins6,InsectMiirotateha,FtrInsectMiirotateha,620,aj95rMzdbSbvZy9A2,NaN,NaN,NaN,NaN,Agrias_Butterfly,Agrias_Butterfly,[[Agrias claudina sardanapalus]],[Agrias claudina sardanapalus]
1,69,ant,80,On rotten turnips,Any weather,0,0,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,Black,White,Ins26,InsectAri,FtrInsectAri,588,QZpmczZu4hW2a4Rpv,NaN,NaN,NaN,NaN,Ant,Ant,[Unknown],Unknown
2,14,Atlas moth,3000,On trees (any kind),Any weather,20,5,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,NaN,NaN,NaN,NaN,NaN,NaN,7 PM – 4 AM,7 PM – 4 AM,7 PM – 4 AM,Orange,Yellow,Ins10,InsectYonagunisan,FtrInsectYonagunisan,652,u2GhYQJXDCQKp7AQ8,NaN,NaN,NaN,NaN,Atlas_Moth,Atlas_Moth,[Attacus atlas],Attacus atlas
3,68,bagworm,600,Shaking trees (hardwood or cedar only),Any weather,0,50,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,All day,Brown,Blue,Ins36,InsectMinomushi,FtrInsectMinomushi,622,QvxgCm82JqHsDknY4,NaN,NaN,NaN,NaN,Bagworm,Bagworm,[Metura elongatus],Metura elongatus
4,34,banded dragonfly,4500,Flying near water,Any except rain,50,7,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,8 AM – 5 PM,NaN,NaN,NaN,NaN,NaN,NaN,8 AM – 5 PM,8 AM – 5 PM,Black,Yellow,Ins24,InsectOniyanma,FtrInsectOniyanma,635,pCFep58D6QusMSvR7,NaN,NaN,NaN,NaN,Banded_Dragonfly,Banded_Dragonfly,[[Anotogaster sieboldii]],[Anotogaster sieboldii]


In [25]:
critters_acnh_sci.to_csv("acnh_critters_sci_name.csv")